# Predicting Length of Stay - Data EDA
Xiaoyu Yang
<br>
Created on 4/7/24

In [27]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 2000)
pd.options.mode.copy_on_write = True

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import datetime 

### Initial inspection of the overall structure of data
Observations from an initial inspection:
1. The data seems to be pulling from four sources: clients, appointments, episode of care (EOC), and providers
2. The data seems to be at client + appointment level, but the prediction is at client level. There should be some aggregation of information at client level
3. There is not a unique identifier for clients. I need to create one
4. There is not an existing target column in the data. I need to create a target variable using appointment data using the combination of appointment dates, appointment status and discharge status
5. Some columns have wrong data types. I need to correct them for further processing to create modeling-ready data set
6. There are test rows in the data. I need to exclude them from any EDA or modeling

In [28]:
df = pd.read_csv('./data/analytics_case_study_raw_data (3).csv')

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9100 entries, 0 to 9099
Data columns (total 69 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   APPOINTMENT_ID_PRIMARY_KEY                                9100 non-null   int64  
 1   APPOINTMENT_DESCRIPTION_LOW_LEVEL_X                       9100 non-null   object 
 2   APPOINTMENT_DATE_X                                        9100 non-null   object 
 3   APPOINTMENT_ATTENDANCE_STATUS_NAME_X                      9100 non-null   object 
 4   EOC_KEY_X                                                 9100 non-null   int64  
 5   APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X                      9100 non-null   object 
 6   Assigned PT                                               9100 non-null   object 
 7   Assigned CS                                               9100 non-null   object 
 8   Assigned CD       

In [30]:
df.isnull().sum()

APPOINTMENT_ID_PRIMARY_KEY                                     0
APPOINTMENT_DESCRIPTION_LOW_LEVEL_X                            0
APPOINTMENT_DATE_X                                             0
APPOINTMENT_ATTENDANCE_STATUS_NAME_X                           0
EOC_KEY_X                                                      0
APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X                           0
Assigned PT                                                    0
Assigned CS                                                    0
Assigned CD                                                    0
EOC_SALESFORCE_OPPORTUNITY_ID_CRM                              0
EOC_INTERESTED_IN_FAMILY_THERAPY_CRM                         381
EOC_FAMILY_THERAPY_SCHEDULED_DURING_BPS_CRM                    0
EOC_FAMILY_THERAPY_NOT_SCHEDULED_DURING_BPS_REASON_CRM      7777
EOC_SCHEDULED_INDIVIDUAL_THERAPY_HOURS_PRODUCT                 0
EOC_SCHEDULED_FAMILY_THERAPY_HOURS_PRODUCT                     0
EOC_REFERRAL_SOURCE_TYPE_

In [31]:
df.head()

,APPOINTMENT_ID_PRIMARY_KEY,APPOINTMENT_DESCRIPTION_LOW_LEVEL_X,APPOINTMENT_DATE_X,APPOINTMENT_ATTENDANCE_STATUS_NAME_X,EOC_KEY_X,APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X,Assigned PT,Assigned CS,Assigned CD,EOC_SALESFORCE_OPPORTUNITY_ID_CRM,EOC_INTERESTED_IN_FAMILY_THERAPY_CRM,EOC_FAMILY_THERAPY_SCHEDULED_DURING_BPS_CRM,EOC_FAMILY_THERAPY_NOT_SCHEDULED_DURING_BPS_REASON_CRM,EOC_SCHEDULED_INDIVIDUAL_THERAPY_HOURS_PRODUCT,EOC_SCHEDULED_FAMILY_THERAPY_HOURS_PRODUCT,EOC_REFERRAL_SOURCE_TYPE_CRM,EOC_REFERRAL_SOURCE_SUBTYPE_CRM,EOC_REFERRAL_SOURCE_STATE_CRM,EOC_OUTREACH_REFERRAL_OWNER_CRM,EOC_HAS_CARE_CENTER_ACCESS_PRODUCT,EOC_NUMBER_OF_CARE_CENTER_ACCOUNTS_WITH_ACCESS_PRODUCT,CLIENT_DATE_OF_BIRTH_CRM,CLIENT_RACE_ETHNICITY_NAME_CRM,CLIENT_RELIGION_NAME_CRM,CLIENT_PREFERRED_LANGUAGE_CRM,CLIENT_GENDER_CRM,CLIENT_PREFERRED_PRONOUN_CRM,CLIENT_PREFERRED_NAME_IS_SAFE_CRM,CLIENT_REFERRAL_SOURCE_CONTACT_NAME_CRM,CLIENT_SEX_EHR,EOC_PROGRAM_TYPE_MH_OR_SUD_CRM,EOC_OPPORTUNITY_CONFIRMED_LEAD_SOURCE_TYPE_CRM,EOC_PAYMENT_PLAN_IS_NEEDED_CRM,EOC_ADMISSION_PENDING_REASON_CRM,EOC_REASON_FOR_DISCHARGE_CRM,CLIENT_ACCOUNT_SOURCE_TYPE_CRM,CLIENT_ACCOUNT_TYPE_NAME_CRM,CLIENT_ACCOUNT_IS_TEST_ACCOUNT_CRM,EOC_PROGRAM_AND_CARE_GROUP_CODE_PRODUCT,EOC_CARE_GROUP_CODE_PRODUCT,EOC_CARE_PROGRAM_ADOL_VS_YA_PRODUCT,EOC_INSURANCE_PRIMARY_OR_SECONDARY_CRM,EOC_INSURANCE_PLAN_TYPE_CRM,EOC_INSURANCE_PAYOR_TYPE_CRM,EOC_INSURANCE_PLAN_NAME_CRM,EOC_INSURANCE_CARRIER_NAME_CRM,CLIENT_STATE_CODE_X,CLIENT_ADDRESS_LINE1_X,CLIENT_CITY_X,CLIENT_ZIP_CODE_X,CLIENT_COUNTRY_X,CLIENT_COUNTRY_CODE_X,CLIENT_PHONE_X,CLIENT_EMAIL_X,CLIENT_FULL_NAME_NO_PREFERRED_NAME_X,CLIENT_FULL_NAME_WITH_PREFERRED_NAME_X,Appointment provider,Appointment provider supervisor,Appointment Provider Race,Appointment Provider employment type,Appointment Provider Charlie Health tenure (years),Appointment Provider Years of experience (years),Appointment Provider Gender,Assigned Primary Therapist Race,Assigned Primary Therapist employment type,Assigned Primary Therapist Charlie Health tenure (years),Assigned Primary Therapist Years of experience (years),Assigned Primary Therapist Gender,Session Notes
0,0,Group Session,12/19/2023,Attended,11138868,IOP,PT 34,CS 7,CD 3,208-39-8583,False,FALSE,NaN,1,0,Outpatient Practice,NaN,Arkansas,Andi Vilaboy,True,1,5/18/2001,White,No Religious Affiliation,English,Female,He/Him,False,Sean Gutierrez,F,Mental Health,Professional Referral,False,NaN,Routine,Professional Referral,Client - Patient,False,ADOL-DLGO17,DGO79,ADOL,Primary,PPO,Commercial,Randall Miller,Jennifer Kramer,Oregon,21109 Barrera Hills Apt. 884,Powersmouth,79517.0,United States,ID,507.622.8033,berrydawn@example.com,Karen Perez,Timothy Smith,Group Facilitator 31,Group Supervisor 7,White,1099,1.0,9.0,Male,unknown,employee_full_time,1,4,Male,The session revolved around the client's anxie...
1,2,Group Session,1/15/2024,Incomplete,10422996,IOP,PT 22,CS 5,CD 2,819-55-7947,True,FALSE,NaN,1,1,Outpatient Practice,NaN,Minnesota,Amanda Davison,True,0,11/21/2009,White,No Religious Affiliation,English,Female,He/Him,False,Victor Bass,M,Mental Health,Internet,False,NaN,Routine,Internet,Client - Patient,False,ADOL-CFTO24,DGY55,ADOL,Primary,OA,Commercial,Pamela Horton,Melissa Newton,Virginia,711 Mclaughlin Via,Hillfort,25045.0,United States,GR,242-447-4376,nicholeking@example.org,Nicholas Mccormick,Benjamin Jackson,Group Facilitator 1,Group Supervisor 1,White,1099,0.0,2.0,Female,Other Race,employee_full_time,2,10,Male,NaN
2,4,Group Session,11/15/2023,Attended,11893825,IOP,PT 42,CS 9,CD 3,873-76-1907,True,FALSE,NaN,1,0,NaN,Hospital System,Minnesota,D Becker,True,1,6/5/2000,Other Race,Christianity,English,Non-Binary,She/Her,False,NaN,Female,Mental Health,Professional Referral,False,NaN,Routine,Professional Referral - Revalidate,Client - Patient,False,ADOL-POCY2,DGO43,YA,Primary,OA+,Commercial,Amy Brown,Caroline Dixon,Alabama,669 Williams Fort,Brownmouth,51288.0,United States,PE,+1-974-547-9409x551,gregoryevans@example.net,Dale Bowers,Nancy

### Data processing
I will first probe into the data by each source and create aggregated features either within the data source or across different data sources

In [32]:
# filter data for non-test records
df = df[df['CLIENT_ACCOUNT_IS_TEST_ACCOUNT_CRM'] == False].reset_index()

# create a client_id using client email + birthday
df['client_id'] = df['CLIENT_EMAIL_X'].astype('str') + df['CLIENT_DATE_OF_BIRTH_CRM'].astype('str')
df['CLIENT_DATE_OF_BIRTH_CRM'] = pd.to_datetime(df['CLIENT_DATE_OF_BIRTH_CRM'])
df['APPOINTMENT_DATE_X'] = pd.to_datetime(df['APPOINTMENT_DATE_X'])

In [33]:
# split the dataset into 4 subsets for more efficient data EDA
cli_cols = [col for col in df.columns if col.startswith('CLIENT')]
app_cols = [col for col in df.columns if col.startswith('APPOINTMENT')]
eoc_cols = [col for col in df.columns if col.startswith('EOC')]
prd_cols = [col for col in df.columns if col not in cli_cols + app_cols + eoc_cols + ['client_id']]

cli = df[['client_id'] + cli_cols + prd_cols]
app = df[['client_id'] + app_cols + eoc_cols]

In [53]:
df.APPOINTMENT_DATE_X.min(), df.APPOINTMENT_DATE_X.max()

(Timestamp('2023-10-01 00:00:00'), Timestamp('2024-03-05 00:00:00'))

In [56]:
(df.APPOINTMENT_DATE_X.max() - df.APPOINTMENT_DATE_X.min()) // np.timedelta64(1, 'W')

22

#### Client data EDA
1. Probe into demographic features such as age, gender, religion, language preferences and source of referral
2. For geographic features, they may be useful when combined with other data sources. More to explore later

In [34]:
# Although there are 8633 records at client and appointment level, there are only 887 unique clients, and they can be captured by client email + birthday. I created a client_id with client email + birthday (in string format)
cli.shape, cli.drop_duplicates().shape, cli.drop_duplicates(subset = ['CLIENT_EMAIL_X', 'CLIENT_DATE_OF_BIRTH_CRM']).shape

((8633, 40), (8633, 40), (887, 40))

In [35]:
cli.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8633 entries, 0 to 8632
Data columns (total 40 columns):
 #   Column                                                    Non-Null Count  Dtype         
---  ------                                                    --------------  -----         
 0   client_id                                                 8633 non-null   object        
 1   CLIENT_DATE_OF_BIRTH_CRM                                  8633 non-null   datetime64[ns]
 2   CLIENT_RACE_ETHNICITY_NAME_CRM                            8633 non-null   object        
 3   CLIENT_RELIGION_NAME_CRM                                  8624 non-null   object        
 4   CLIENT_PREFERRED_LANGUAGE_CRM                             8531 non-null   object        
 5   CLIENT_GENDER_CRM                                         8540 non-null   object        
 6   CLIENT_PREFERRED_PRONOUN_CRM                              8633 non-null   object        
 7   CLIENT_PREFERRED_NAME_IS_SAFE_CRM         

In [36]:
cli.isnull().sum()

client_id                                                      0
CLIENT_DATE_OF_BIRTH_CRM                                       0
CLIENT_RACE_ETHNICITY_NAME_CRM                                 0
CLIENT_RELIGION_NAME_CRM                                       9
CLIENT_PREFERRED_LANGUAGE_CRM                                102
CLIENT_GENDER_CRM                                             93
CLIENT_PREFERRED_PRONOUN_CRM                                   0
CLIENT_PREFERRED_NAME_IS_SAFE_CRM                              0
CLIENT_REFERRAL_SOURCE_CONTACT_NAME_CRM                     1384
CLIENT_SEX_EHR                                                 0
CLIENT_ACCOUNT_SOURCE_TYPE_CRM                                 0
CLIENT_ACCOUNT_TYPE_NAME_CRM                                   0
CLIENT_ACCOUNT_IS_TEST_ACCOUNT_CRM                             0
CLIENT_STATE_CODE_X                                            0
CLIENT_ADDRESS_LINE1_X                                       113
CLIENT_CITY_X            

In [37]:
# give the correct data type to birthday and zip code columns
cli['CLIENT_DATE_OF_BIRTH_CRM'] = pd.to_datetime(cli['CLIENT_DATE_OF_BIRTH_CRM'])
cli['CLIENT_ZIP_CODE_X'] = cli['CLIENT_ZIP_CODE_X'].apply(lambda x: "{:.0f}".format(x) if not pd.isnull(x) else x).astype('str').apply(lambda x: x.zfill(5))

In [38]:
# calculate age using the current date
cli['age'] = (datetime.datetime.today() - cli.CLIENT_DATE_OF_BIRTH_CRM) // np.timedelta64(1, 'Y')

In [39]:
# EHR is the most reliable source of gender, but it is also useful to create a flag to indicate if the client has special gender identity
cli.groupby('CLIENT_SEX_EHR').client_id.nunique()

CLIENT_SEX_EHR
F                 442
Female             68
Intersex/Other      3
M                 248
Male               25
N                  74
Unknown            27
Name: client_id, dtype: int64

In [40]:
cli.groupby('CLIENT_GENDER_CRM').client_id.nunique()

CLIENT_GENDER_CRM
Female                                      511
Gender Fluid                                  3
Gender Neutral                               29
Genderqueer Gender Nonconforming Neither      7
Male                                        286
Non-Binary                                   21
Other                                         1
Transgender                                  17
Name: client_id, dtype: int64

In [41]:
cli.groupby(['CLIENT_SEX_EHR', 'CLIENT_GENDER_CRM']).client_id.nunique()

CLIENT_SEX_EHR  CLIENT_GENDER_CRM                       
F               Female                                      310
                Gender Neutral                                5
                Genderqueer Gender Nonconforming Neither      3
                Male                                        119
                Non-Binary                                    1
                Other                                         1
                Transgender                                   1
Female          Female                                       30
                Gender Fluid                                  2
                Gender Neutral                                4
                Male                                         19
                Non-Binary                                    8
                Transgender                                   2
Intersex/Other  Female                                        1
                Male                           

In [42]:
cli['gender'] = np.where(
    (cli.CLIENT_SEX_EHR.isin(['F', 'Female'])) & ((cli.CLIENT_GENDER_CRM) == 'Female'), 'Female', np.where(
        (cli.CLIENT_SEX_EHR.isin(['M', 'Male'])) & ((cli.CLIENT_GENDER_CRM) == 'Male'), 'Male', 'Special'
    )
)

In [43]:
# Pick top 2 ethinicies and create a race feature with less levels
cli.groupby('CLIENT_RACE_ETHNICITY_NAME_CRM').client_id.nunique()

CLIENT_RACE_ETHNICITY_NAME_CRM
2 or more                                     40
American Indian or Alaska Native               7
Asian                                         14
Black or African American                    125
Hispanic or Latino                             1
Native Hawaiian or Other Pacific Islander      3
Other Race                                    98
White                                        597
unknown                                        2
Name: client_id, dtype: int64

In [44]:
cli['race'] = np.where(
    cli.CLIENT_RACE_ETHNICITY_NAME_CRM == 'White', 'White', np.where(
        cli.CLIENT_RACE_ETHNICITY_NAME_CRM == 'Black or Africa America', 'Black', 'Other'
    )
)

In [45]:
# Create a binary variable to indicate if a client has religious belief
cli.groupby('CLIENT_RELIGION_NAME_CRM').client_id.nunique()

CLIENT_RELIGION_NAME_CRM
Agnostic                     11
Atheist                       4
Baptist                       5
Buddhism                      4
Catholic                     35
Christianity                166
Episcopal                    41
Hinduism                      1
Jehova's Witness              4
Jewish                       21
Judaism                       2
Lutheran                      1
Mormon                        5
Muslim                        2
No Religious Affiliation    562
Nondenominational            11
Pagan                         3
Protestant                    3
Unknown Religion              4
Unstated Religion             1
Name: client_id, dtype: int64

In [46]:
cli['religion_ind'] = np.where(cli.CLIENT_RELIGION_NAME_CRM == 'No Religious Affiliation', 'N', 'Y')

In [47]:
# Create a binary feature to indicate if a client prefers Spanish
cli.groupby('CLIENT_PREFERRED_LANGUAGE_CRM').client_id.nunique()

CLIENT_PREFERRED_LANGUAGE_CRM
English    843
Spanish     37
Name: client_id, dtype: int64

In [48]:
cli['spanish_ind'] = np.where(cli.CLIENT_PREFERRED_LANGUAGE_CRM == 'Spanish', 'Y', 'N')

In [49]:
# combine the processed columns for modeling. Below are the demographic variables for modeling. I will examine geographic features later
df_cli = cli[['client_id', 'age', 'gender', 'race', 'religion_ind', 'spanish_ind']]

In [50]:
df_cli.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8633 entries, 0 to 8632
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   client_id     8633 non-null   object
 1   age           8633 non-null   int64 
 2   gender        8633 non-null   object
 3   race          8633 non-null   object
 4   religion_ind  8633 non-null   object
 5   spanish_ind   8633 non-null   object
dtypes: int64(1), object(5)
memory usage: 404.8+ KB


In [51]:
df_cli.describe()

,age
count,8633.000000
mean,20.082358
std,4.368966
min,12.000000
25%,17.000000
50%,20.000000
75%,23.000000
max,34.000000


#### Appointment + EOC data EDA

In [52]:
app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8633 entries, 0 to 8632
Data columns (total 32 columns):
 #   Column                                                  Non-Null Count  Dtype         
---  ------                                                  --------------  -----         
 0   client_id                                               8633 non-null   object        
 1   APPOINTMENT_ID_PRIMARY_KEY                              8633 non-null   int64         
 2   APPOINTMENT_DESCRIPTION_LOW_LEVEL_X                     8633 non-null   object        
 3   APPOINTMENT_DATE_X                                      8633 non-null   datetime64[ns]
 4   APPOINTMENT_ATTENDANCE_STATUS_NAME_X                    8633 non-null   object        
 5   APPOINTMENT_DESCRIPTION_HIGH_LEVEL_X                    8633 non-null   object        
 6   EOC_KEY_X                                               8633 non-null   int64         
 7   EOC_SALESFORCE_OPPORTUNITY_ID_CRM                       8633

#### Provider data EDA

### Create and save the modeling-ready data set
Before any further data EDA between the target variable and eligible features, I'll randomly split 20% of data into a test set and set it aside. This set will be used to evaluate the model on out samples